# Refrences

In [ ]:
# This code was inspired by Kero Ashraf's work. Here is a link to the orgnial Kaggle file. https://www.kaggle.com/code/kirollosashraf/human-action-recognition-har/notebook

In [ ]:
'''
This file is inteded to help futrue iterations by giving the devloper a jumpping of point.
We could not find an adiquite dataset to produce a model to detect human actions in a classroom ie poiting at the borad, rasing hand, etc
Datasets we do not recomend
- UCF101
- Charades
- Something-Something (V2)
'''

# pip install

In [ ]:
!pip install numpy
!pip install pandas
!pip install tensorflow
!pip install tqdm
!pip install Pillow
!pip install seaborn
!pip install matplotlib

# Importing libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import glob
import random
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Activation, Dropout, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator # Updated import statement

from tqdm import tqdm

from PIL import Image

from tensorflow.keras.utils import to_categorical

import seaborn as sns
import matplotlib.image as img
import matplotlib.pyplot as plt

# 3| Getting the path and Loading the data

In [ ]:
google_drive_path = '/content/drive/MyDrive/Human Action Recognition/'

train_data = pd.read_csv(google_drive_path + "Training_set.csv")
test_data = pd.read_csv(google_drive_path + "Testing_set.csv")

In [ ]:
train_fol = glob.glob(google_drive_path + "train/*")
test_fol = glob.glob(google_drive_path + "test/*")

In [ ]:
train_data

In [ ]:
train_data.label.value_counts()

In [ ]:
import plotly.express as px
HAR = train_data.label.value_counts()
fig = px.pie(train_data, values=HAR.values, names=HAR.index, title='Distribution of Human Activity')
fig.show()

In [ ]:
filename = train_data['filename']

situation = train_data['label']

In [ ]:
filename

# 4| Making function that take random path and display the image

In [ ]:
def displaying_random_images():
    num = random.randint(1,10000)
    imgg = "Image_{}.jpg".format(num)
    train = google_drive_path + "train/"
    if os.path.exists(train+imgg):
        testImage = img.imread(train+imgg)
        plt.imshow(testImage)
        plt.title("{}".format(train_data.loc[train_data['filename'] == "{}".format(imgg), 'label'].item()))

    else:
        print("File Path not found \nSkipping the file!!")

In [ ]:
displaying_random_images()

In [ ]:
displaying_random_images()

In [ ]:
displaying_random_images()

In [ ]:
displaying_random_images()

# 5| Data preprocessing

In [ ]:
img_data = []
img_label = []
length = len(train_fol)
for i in (range(len(train_fol)-1)):
    t = google_drive_path + 'train/' + filename[i]
    temp_img = Image.open(t)
    img_data.append(np.asarray(temp_img.resize((160,160))))
    img_label.append(situation[i])

In [ ]:
img_shape= (160,160,3)

In [ ]:
iii = img_data
iii = np.asarray(iii)
type(iii)

In [ ]:
y_train = to_categorical(np.asarray(train_data["label"].factorize()[0]))
print(y_train[0])

# 6| Make an CNN model

In [ ]:
efficientnet_model = Sequential()

model = tf.keras.applications.EfficientNetB7(include_top=False,
                                            input_shape=(160,160,3),
                                            pooling ="avg",classes=15,
                                             weights="imagenet")

for layer in model.layers:
    layer.trainable=False


efficientnet_model.add(model)
efficientnet_model.add(Flatten())
efficientnet_model.add(Dense(512,activation="relu"))
efficientnet_model.add(Dense(15,activation="softmax"))

In [ ]:
efficientnet_model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [ ]:
efficientnet_model.summary()

In [ ]:
history = efficientnet_model.fit(iii,y_train,epochs=40)

In [ ]:
losses = history.history["loss"]
plt.plot(losses)

In [ ]:
acc = history.history['accuracy']
plt.plot(acc)

# 7| Model predictions

In [ ]:
def read_img(fn):
    img = Image.open(fn)
    return np.asarray(img.resize((160,160)))

In [ ]:
def test_predict(test_image):
    result = efficientnet_model.predict(np.asarray([read_img(test_image)]))

    itemindex = np.where(result==np.max(result))
    prediction = itemindex[1][0]
    print("probability: "+str(np.max(result)*100) + "%\nPredicted class : ", prediction)

    image = img.imread(test_image)
    plt.imshow(image)
    plt.title(prediction)

In [ ]:
test_predict(google_drive_path + "test/Image_1001.jpg")

In [ ]:
test_predict(google_drive_path + "test/Image_101.jpg")

In [ ]:
test_predict(google_drive_path + "test/Image_1056.jpg")

In [ ]:
test_predict(google_drive_path + "test/Image_1024.jpg")